In [1]:
from ase.io import read
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
materials = read('OMDB-GAP1_v1.1/structures.xyz', index=':')

In [3]:
materials[0]

Atoms(symbols='C30H56Mo2N8O4P4', pbc=True, cell=[[8.487989, 0.0, -0.505023], [-3.031581, 7.876696, -1.51585], [0.0, 0.0, 14.584]])

In [4]:
df = pd.read_csv('OMDB-GAP1_v1.1/CODids.csv',encoding = 'unicode_escape',names=['ids'])
df2= pd.read_csv('OMDB-GAP1_v1.1/bandgaps.csv',encoding = 'unicode_escape')
df['bandgaps']=df2
# df=df.head(1000)

In [5]:
featurelist= ['_chemical_formula_sum', "_chemical_formula_weight",  "_space_group_IT_number", 
"_symmetry_cell_setting",  "_symmetry_space_group_name_H-M",  "_cell_angle_alpha",  "_cell_angle_beta", 
"_cell_angle_gamma",  "_cell_formula_units_Z",  "_cell_length_a",  "_cell_length_b",  "_cell_length_c", 
"_cell_measurement_reflns_used",  "_cell_measurement_temperature",  "_cell_measurement_theta_max",  
"_cell_measurement_theta_min",  "_cell_volume",  "_diffrn_radiation_wavelength",  
"_diffrn_reflns_av_R_equivalents",  "_diffrn_reflns_av_sigmaI/netI",  "_diffrn_reflns_limit_h_max",  
"_diffrn_reflns_limit_h_min",  "_diffrn_reflns_limit_k_max",  "_diffrn_reflns_limit_k_min",  
"_diffrn_reflns_limit_l_max",  "_diffrn_reflns_limit_l_min",  "_diffrn_reflns_number",  
"_diffrn_reflns_theta_full",  "_diffrn_reflns_theta_max",  "_diffrn_reflns_theta_min",  
"_exptl_absorpt_coefficient_mu",  "_exptl_absorpt_correction_T_max",  "_exptl_absorpt_correction_T_min"]

  
# choosing features 
#featurelist = ['_chemical_formula_sum','_chemical_formula_weight',"_symmetry_space_group_name_H-M"]
for x in featurelist:
    stringlist=['_chemical_formula_sum', "_symmetry_space_group_name_H-M"]
    if x in stringlist:
        df[x]= ''
    else:
        df[x]=-100.1

In [6]:
# get info 
for x in df['ids']:
    id= str(x)
    index=df.loc[df['ids'] == x].index[0]
    url=requests.get('http://www.crystallography.net/cod/'+id+'.cif')
    htmltext = url.text
    for y in featurelist:
        matched_lines = [line for line in htmltext.split('\n') if y in line]
        if len(matched_lines)>0:
            if y=="_chemical_formula_sum" or y=="_symmetry_space_group_name_H-M":
                matched_lines= matched_lines[0].split("'")
                # df.set_value(index, y, matched_lines[1])
                df.loc[index,y] = matched_lines[1]
            else:
                matched_lines= matched_lines[0].split(" ")
                k=matched_lines[-1]
                if '(' in k:
                    k= k.split('(')[0]
                if y!="_symmetry_cell_setting":
                    try:
                        k=float(k)
                    except ValueError:
                        k=-100
                if y=="_symmetry_cell_setting":
                    if k== "monoclinic":
                        k=1.0
                    elif k=="triclinic":
                        k=3.0
                    elif k=="orthorhombic":
                        k=5.0
                    else:
                        k=7.0
                # df.set_value(index, y, k)
                df.loc[index,y] = k

In [ ]:
df.to_csv('final_db.csv', sep='\t')